<a href="https://colab.research.google.com/github/MengOonLee/Web_scraping/blob/master/Tutorial/Scrapy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scrapy

In [ ]:
%%bash

pip install --no-cache-dir -qU scrapy

In [1]:
import scrapy
import requests
url = "https://quotes.toscrape.com/tag/humor/"
html = requests.get(url).content

sel = scrapy.Selector(text=html)
quote = sel.css('div.quote')
author = quote.xpath('span/small/text()').get()
print('Author:', author)
text = quote.css('span.text::text').get()
print('Text:', text)

next_page = sel.css('li.next a::attr(href)').get()
print('Next page:', next_page)

Author: Jane Austen
Text: “The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”
Next page: /tag/humor/page/2/


In [2]:
%%bash
scrapy startproject tutorial

New Scrapy project 'tutorial', using template directory '/Work/venv/lib/python3.8/site-packages/scrapy/templates/project', created in:
    /Work/Web_scraping/Tutorial/tutorial

You can start your first spider with:
    cd tutorial
    scrapy genspider example example.com


In [3]:
%%writefile ./tutorial/quotes_spider.py
import scrapy

class QuotesSpider(scrapy.Spider):
    name = 'quotes'
    start_urls = [
        'https://quotes.toscrape.com/tag/humor/'
    ]

    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'author': quote.xpath('span/small/text()').get(),
                'text': quote.css('span.text::text').get()
            }

        next_page = response.css('li.next a::attr(href)').get()
        if next_page is not None:
            yield response.follow(next_page, self.parse)

Writing ./tutorial/quotes_spider.py


In [4]:
%%bash

scrapy runspider ./tutorial/quotes_spider.py -O ./data/quotes.jl

2023-09-10 14:51:34 [scrapy.utils.log] INFO: Scrapy 2.10.1 started (bot: scrapybot)
2023-09-10 14:51:34 [scrapy.utils.log] INFO: Versions: lxml 4.9.3.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.8.10 (default, May 26 2023, 14:05:08) - [GCC 9.4.0], pyOpenSSL 23.2.0 (OpenSSL 3.1.2 1 Aug 2023), cryptography 41.0.3, Platform Linux-6.2.0-31-generic-x86_64-with-glibc2.29
2023-09-10 14:51:34 [scrapy.addons] INFO: Enabled addons:
[]
2023-09-10 14:51:34 [scrapy.crawler] INFO: Overridden settings:
{'SPIDER_LOADER_WARN_ONLY': True}
2023-09-10 14:51:34 [py.warnings] WARNING: /Work/venv/lib/python3.8/site-packages/scrapy/utils/request.py:248: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility re

In [ ]:
%%writefile ./Tutorial/Tutorial/spiders/quotes_spider.py
import scrapy

class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        "https://quotes.toscrape.com/page/1/",
        "https://quotes.toscrape.com/page/2/"
    ]

    def parse(self, response):
        page = response.url.split("/")[-2]
        filename = f'./data/quotes-{page}.html'
        with open(filename, 'wb') as f:
            f.write(response.body)

In [ ]:
%%writefile ./Tutorial/run.sh
#!/bin/bash

scrapy crawl quotes

In [ ]:
from scrapy import Selector
import requests

url = "https://quotes.toscrape.com/page/1/"
html = requests.get(url).content

sel = Selector(text=html)
# CSS
title = sel.css('title::text')
print(f"css title: {title}")
print(title.re(r'(\w+) to (\w+)'))

# XPath
title = sel.xpath('//title/text()')
print(f"xpath title: {title}")

In [ ]:
from scrapy import Selector
import requests

url = "https://quotes.toscrape.com/"
html = requests.get(url).content

sel = Selector(text=html)

for quote in sel.css('div.quote'):
    text = quote.css('span.text::text').extract_first()
    author = quote.css('small.author::text').extract_first()
    tags = quote.css('div.tags a.tag::text').extract()
    print(dict(text=text, author=author, tags=tags))

In [ ]:
%%writefile ./Tutorial/Tutorial/spiders/quotes_spider.py
import scrapy

class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
      'https://quotes.toscrape.com/page/1/',
      'https://quotes.toscrape.com/page/2/'
    ]

    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').get(),
                'author': quote.css('small.author::text').get(),
                'tags': quote.css('div.tags a.tag::text').getall()
            }

In [ ]:
%%writefile ./Tutorial/run.sh
#!/bin/bash

scrapy crawl quotes

### Storing the scraped data

In [ ]:
%%writefile ./Tutorial/run.sh
#!/bin/bash

scrapy crawl quotes -O ./data/quotes.json

In [ ]:
%%writefile ./Tutorial/run.sh
#!/bin/bash

scrapy crawl quotes -o ./data/quotes.jl

## Following links

In [ ]:
%%bash
scrapy shell "https://quotes.toscrape.com"
response.css("li.next a").get()
response.css("li.next a::attr(href)").get()
response.css("li.next a").attrib["href"]

In [ ]:
%%writefile ./tutorial/tutorial/spiders/quotes_spider.py
import scrapy

class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
      'https://quotes.toscrape.com/page/1/'
    ]

    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').get(),
                'author': quote.css('small.author::text').get(),
                'tags': quote.css('div.tags a.tag::text').getall()
            }
        next_page = response.css('li.next a::attr(href)').get()
        if next_page is not None:
            next_page = response.urljoin(next_page)
            yield scrapy.Request(next_page, callback=self.parse)

In [ ]:
%%bash
cd tutorial
rm -rf tutorial/quotes.jl
scrapy crawl quotes -o quotes.jl

### Supports relative URLs directly

In [ ]:
%%writefile ./tutorial/tutorial/spiders/quotes_spider.py
import scrapy

class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        'https://quotes.toscrape.com/page/1/'
    ]

    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').get(),
                'author': quote.css('small.author::text').get(),
                'tags': quote.css('div.tags a.tag::text').getall()
            }
        next_page = response.css('li.next a::attr(href)').get()
        if next_page is not None:
            yield response.follow(next_page, callback=self.parse)

In [ ]:
%%bash
cd tutorial
rm -rf tutorial/quotes.jl
scrapy crawl quotes -o quotes.jl

In [ ]:
%%writefile ./tutorial/tutorial/spiders/quotes_spider.py
import scrapy

class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        'https://quotes.toscrape.com'
    ]

    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').get(),
                'author': quote.css('small.author::text').get(),
                'tags': quote.css('div.tags a.tag::text').getall()
            }
        for href in response.css('li.next a::attr(href)'):
            yield response.follow(href, callback=self.parse)

In [ ]:
%%bash
cd tutorial
rm -rf tutorial/quotes.jl
scrapy crawl quotes -o quotes.jl

In [ ]:
%%writefile ./tutorial/tutorial/spiders/quotes_spider.py
import scrapy

class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        'https://quotes.toscrape.com'
    ]

    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').get(),
                'author': quote.css('small.author::text').get(),
                'tags': quote.css('div.tags a.tag::text').getall()
            }
        for a in response.css('ul.pager li.next a'):
            yield response.follow(a, callback=self.parse)

In [ ]:
%%bash
cd tutorial
rm -rf tutorial/quotes.jl
scrapy crawl quotes -o quotes.jl

### Create multiple requests from an iterable

In [ ]:
%%writefile ./tutorial/tutorial/spiders/quotes_spider.py
import scrapy

class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        'https://quotes.toscrape.com'
    ]

    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').get(),
                'author': quote.css('small.author::text').get(),
                'tags': quote.css('div.tags a.tag::text').getall()
            }
        anchors = response.css('ul.pager li.next a')
        yield from response.follow_all(anchors, callback=self.parse)

In [ ]:
%%bash
cd tutorial
rm -rf tutorial/quotes.jl
scrapy crawl quotes -o quotes.jl

In [ ]:
%%writefile ./tutorial/tutorial/spiders/quotes_spider.py
import scrapy

class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = [
        'https://quotes.toscrape.com'
    ]

    def parse(self, response):
        for quote in response.css('div.quote'):
            yield {
                'text': quote.css('span.text::text').get(),
                'author': quote.css('small.author::text').get(),
                'tags': quote.css('div.tags a.tag::text').getall()
            }
        yield from response.follow_all(css='ul.pager li.next a', callback=self.parse)

In [ ]:
%%bash
cd tutorial
rm -rf tutorial/quotes.jl
scrapy crawl quotes -o quotes.jl

## More patterns

In [ ]:
%%writefile ./tutorial/tutorial/spiders/author_spider.py
import scrapy

class AuthorSpider(scrapy.Spider):
    name = 'author'

    start_urls = ['https://quotes.toscrape.com/']

    def parse(self, response):
        author_page_links = response.css('.author + a')